In [ ]:
import os
from subprocess import Popen, PIPE

# Create and change to a directory to run our simulations

In [ ]:
%cd

In [ ]:
!mkdir gromacs_sim_run
os.chdir("gromacs_sim_run")
!pwd
!cp ../BioPhyCourse-main/Day1/05_Gromacs/data/* ./

In [ ]:
!pwd

# Editing simulation cell size and shape

In [ ]:
!gmx editconf -h

In [ ]:
!gmx editconf -f 2YVB.pdb -box 6 6 6 -center 3 3 3 -o centered.pdb -angles 90 90 90

# Generating topology and hydrogen atoms for the x-ray structure

In [ ]:
!gmx pdb2gmx -h

In [ ]:
!gmx pdb2gmx -f centered.pdb -ff amber99sb-ildn -water spce -o protein.pdb -ignh -vsite hydrogens -p topol.top

# Generating water solvent

In [ ]:
!gmx solvate -cp protein.pdb -cs -o water.pdb

In [ ]:
!gmx grompp -f em.mdp -p topol.top -c water.pdb -o ion.tpr

# Generating ions

In [ ]:
!gmx genion -h

In [ ]:
p = Popen("gmx genion -s ion.tpr -conc 0.15 -neutral -o ion.pdb".encode(), shell=True, stdin=PIPE)
p.communicate('13\n'.encode())

# Performing energy minimization

In [ ]:
!gmx grompp -f em.mdp -p topol.top -c ion.pdb -o em.tpr

In [ ]:
!gmx mdrun -s em.tpr -c em.pdb -v

# Performing solvent relaxation

In [ ]:
!gmx grompp -f eq.mdp -p topol.top -c em.pdb -o eq.tpr -r em.pdb -maxwarn 1

In [ ]:
!gmx mdrun -s eq.tpr -c eq.pdb -v -ntomp 2  -pin on

In [ ]:
!gmx grompp -f run.mdp -p topol.top -c eq.pdb -o run.tpr -r eq.pdb -maxwarn 1